# Segmenting and Clustering Neighborhoods in Toronto
## Peer-graded Assignment:  to explore and cluster the neighborhoods in Toronto
### *by Leopoldo Sprandel*

### Importing data from wikipedia

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests as rq

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page= rq.get(wikipedia_link).text

In [3]:
raw_list = page[page.find('wikitable sortable')-15:page.find('/table')]
raw_list = pd.read_html(raw_list, header=0, index_col=None, attrs={"class":"wikitable sortable"})[0]
raw_list.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Wrangling data
(done!) Ignore cells with a borough that is Not assigned  
(done!) Two rows with the same PostalCode will be combined into one row with the neighborhoods separated with a comma  
(done!) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough  
(done!) In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [4]:
postals=raw_list
# Rename the column Postcode by PostalCode
postals.rename(columns={'Postcode':'PostalCode'}, inplace=True)
# Check the columns
postals.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# Ignore cells with a borough that is Not assigned
postals.replace("Not assigned", np.nan, inplace = True)
postals.dropna(subset=["Borough"], axis=0, inplace = True)
postals.reset_index(drop = True, inplace = True)
postals.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,NaN
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [6]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
postals['Neighbourhood'].replace(np.nan, postals['Borough'], inplace= True)
postals.head(10)   

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
# Two rows with the same PostalCode will be combined into one row with the neighborhoods separated with a comma
postals=postals.groupby(['PostalCode','Borough'], as_index=False, sort=False).aggregate(lambda x: ', '.join(x))
postals.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
postals.shape

(103, 3)

## Get the latitude and the longitude coordinates of each neighborhood

In [9]:
geocode=pd.read_csv(('http://cocl.us/Geospatial_data'))
geocode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
postals=postals.merge(geocode, left_on='PostalCode', right_on='Postal Code', how='inner').drop(columns=["Postal Code"])

In [11]:
postals.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [12]:
postals.shape

(103, 5)